# Chapter 25: Error Handling and Debugging

---

## Introduction

No matter how skilled a developer you are, errors are an inevitable part of programming. A missing semicolon, a mistyped variable name, an unexpected network failure—all can cause your code to behave unexpectedly or stop working altogether. How you handle these errors separates robust, user‑friendly applications from fragile ones that leave users frustrated.

In this chapter, you will learn:

- The different **types of errors** in JavaScript: syntax, runtime, and logical.
- How to **catch and handle errors** gracefully using `try...catch`.
- How to **throw your own errors** to signal problems in your code.
- **Debugging techniques** and tools to find and fix bugs efficiently.
- Common JavaScript pitfalls and how to avoid them.
- **Defensive programming** practices that prevent errors before they happen.

By mastering error handling and debugging, you will write more reliable code and spend less time tracking down mysterious bugs.

---

## 25.1 Types of Errors

JavaScript errors generally fall into three categories: syntax errors, runtime errors, and logical errors.

### Syntax Errors

Syntax errors occur when the code violates the grammatical rules of JavaScript. The parser cannot understand the code, so the script fails to execute at all.

```javascript
// Missing closing parenthesis
console.log('Hello World';
// SyntaxError: missing ) after argument list

// Using a reserved keyword as a variable name
let class = 'math';
// SyntaxError: unexpected token
```

Syntax errors are usually caught by the editor or browser immediately, and the script will not run.

### Runtime Errors (Exceptions)

Runtime errors occur during execution when the code attempts to do something illegal, such as calling a non‑function or accessing a property of `undefined`. These are also called **exceptions**.

```javascript
// TypeError: user is undefined
const user = null;
console.log(user.name);

// ReferenceError: variable not declared
console.log(nonExistentVariable);

// RangeError: invalid array length
const arr = new Array(-1);
```

When a runtime error occurs, JavaScript throws an exception. If not caught, it stops script execution and logs an error in the console.

### Logical Errors

Logical errors are the hardest to find. The code runs without crashing, but it produces incorrect results. These errors stem from mistakes in the program's logic.

```javascript
// Should calculate average, but sums twice
function average(numbers) {
    let sum = 0;
    for (let i = 0; i <= numbers.length; i++) { // Off‑by‑one: should be <
        sum += numbers[i];
    }
    return sum / numbers.length;
}
console.log(average([10, 20, 30])); // NaN because last iteration accesses undefined
```

Logical errors require careful debugging and testing to uncover.

---

## 25.2 The `try...catch` Statement

The `try...catch` statement allows you to handle runtime errors gracefully, preventing them from crashing your application.

### Basic Syntax

```javascript
try {
    // Code that may throw an error
    const result = riskyOperation();
    console.log('Success:', result);
} catch (error) {
    // Code to execute if an error is thrown
    console.error('An error occurred:', error.message);
} finally {
    // (Optional) Code that runs regardless of error or success
    console.log('Cleanup operations');
}
```

- **`try` block** – contains the code that might throw an exception.
- **`catch` block** – executed if an exception occurs. It receives the error object as a parameter.
- **`finally` block** – always executes after `try`/`catch`, whether an error occurred or not. Useful for releasing resources, closing connections, etc.

### The Error Object

When an error is thrown, JavaScript passes an error object to the `catch` block. It typically has these properties:

- `name` – the type of error (e.g., `'TypeError'`, `'ReferenceError'`).
- `message` – a human‑readable description.
- `stack` – a stack trace showing where the error occurred (useful for debugging).

```javascript
try {
    undefinedFunction();
} catch (err) {
    console.log(err.name);    // "ReferenceError"
    console.log(err.message); // "undefinedFunction is not defined"
    console.log(err.stack);   // Stack trace
}
```

### Conditional Catch Blocks

You cannot have multiple `catch` blocks, but you can inspect the error type and handle different errors differently.

```javascript
try {
    // some code
} catch (err) {
    if (err instanceof TypeError) {
        console.log('Type error handled');
    } else if (err instanceof RangeError) {
        console.log('Range error handled');
    } else {
        console.log('Unknown error:', err);
        // Maybe rethrow?
    }
}
```

### When to Use `try...catch`

Use `try...catch` for code that you expect might fail under normal circumstances, such as:

- Parsing JSON from an external source.
- Making network requests.
- Working with browser APIs that may not be supported.
- Validating user input.

Do not use `try...catch` to guard against simple programming mistakes that can be fixed during development—fix the code instead.

### The `finally` Block

The `finally` block runs after `try` and `catch`, regardless of whether an error occurred. It is commonly used for cleanup.

```javascript
function fetchData() {
    const connection = openDatabaseConnection();
    try {
        // work with connection
        return connection.query('SELECT * FROM users');
    } catch (err) {
        console.error('Query failed:', err);
        return null;
    } finally {
        connection.close(); // always close the connection
    }
}
```

Even if there is a `return` in `try` or `catch`, the `finally` block executes before the function actually returns.

---

## 25.3 Throwing Errors

You can throw your own errors using the `throw` statement. This is useful when you want to signal that something unexpected happened in your code, especially in functions that validate input or enforce contracts.

### The `throw` Statement

You can throw any value, but it's conventional to throw an instance of `Error` or one of its subclasses.

```javascript
function divide(a, b) {
    if (b === 0) {
        throw new Error('Division by zero is not allowed');
    }
    return a / b;
}

try {
    console.log(divide(10, 0));
} catch (err) {
    console.error(err.message); // "Division by zero is not allowed"
}
```

### Creating Custom Error Types

You can create your own error classes by extending the built‑in `Error` class.

```javascript
class ValidationError extends Error {
    constructor(message) {
        super(message);
        this.name = 'ValidationError';
    }
}

class NetworkError extends Error {
    constructor(message, statusCode) {
        super(message);
        this.name = 'NetworkError';
        this.statusCode = statusCode;
    }
}

function validateUser(user) {
    if (!user.name) {
        throw new ValidationError('Name is required');
    }
    if (user.age < 18) {
        throw new ValidationError('User must be at least 18');
    }
}

try {
    validateUser({ name: 'Alice', age: 16 });
} catch (err) {
    if (err instanceof ValidationError) {
        console.log('Validation failed:', err.message);
    } else {
        console.log('Unexpected error:', err);
    }
}
```

Custom errors make it easier to handle different failure modes and provide richer debugging information.

### Rethrowing Errors

Sometimes you want to catch an error, log it, and then rethrow it to be handled at a higher level.

```javascript
function processData(data) {
    try {
        // risky operation
        return JSON.parse(data);
    } catch (err) {
        console.error('Failed to parse JSON:', err);
        throw err; // rethrow
    }
}

try {
    const result = processData('invalid json');
} catch (err) {
    // handle at top level
    alert('Data processing failed');
}
```

Rethrowing preserves the original error information while allowing you to add context.

---

## 25.4 Debugging Techniques

Debugging is the art of finding and fixing bugs. Modern browsers provide powerful tools to help you inspect code, set breakpoints, and examine state.

### `console.log()` and Friends

The simplest debugging tool is `console.log()`, but there are several other useful console methods:

- `console.log()` – general logging.
- `console.info()` – informational messages.
- `console.warn()` – warnings.
- `console.error()` – errors, often with stack trace.
- `console.table()` – displays arrays/objects as a table.
- `console.time()` / `console.timeEnd()` – measure execution time.
- `console.group()` / `console.groupEnd()` – group related logs.
- `console.trace()` – prints a stack trace.

```javascript
console.time('loop');
for (let i = 0; i < 1000000; i++) {}
console.timeEnd('loop'); // loop: 3.123ms

const users = [{ name: 'Alice', age: 30 }, { name: 'Bob', age: 25 }];
console.table(users);
```

### The `debugger` Statement

Inserting `debugger;` in your code acts as a breakpoint. When the browser's developer tools are open, execution will pause at that line, allowing you to inspect variables and step through code.

```javascript
function complexFunction(x, y) {
    debugger; // pause here
    const result = x * y + 100;
    return result;
}
```

This is a convenient way to set a breakpoint without clicking in the DevTools UI.

### Browser DevTools Debugging

All modern browsers include a full‑featured debugger. Key features:

- **Breakpoints** – set breakpoints in the Sources panel to pause execution.
- **Step over / step into / step out** – control execution line by line.
- **Watch expressions** – monitor the value of specific variables.
- **Call stack** – see the chain of function calls that led to the current point.
- **Scope variables** – inspect local, closure, and global variables.
- **Conditional breakpoints** – break only when a condition is true.

**Example workflow:**

1. Open DevTools (F12) and go to the Sources panel.
2. Find your JavaScript file and click the line number to set a breakpoint.
3. Refresh the page or trigger the function.
4. Execution pauses; hover over variables to see their values or use the console to evaluate expressions.
5. Use the step buttons to advance.

### Breakpoints in Practice

- **Line‑of‑code breakpoints** – simplest, pause at a specific line.
- **DOM breakpoints** – pause when a DOM element is modified (in the Elements panel).
- **Event listener breakpoints** – pause when a specific event (e.g., `click`, `keydown`) fires.
- **XHR/fetch breakpoints** – pause when an AJAX request is made.

### Network Tab

The Network tab shows all network requests made by the page. You can inspect request/response headers, payloads, and timing. This is invaluable when debugging API issues.

### Performance Tab

The Performance tab helps you identify slow code, layout thrashing, and memory leaks by recording a profile of your application's runtime.

### Debugging Memory Leaks

Use the Memory tab to take heap snapshots and compare them to find objects that are not being garbage collected.

---

## 25.5 Common JavaScript Errors

Knowing the most frequent errors and their causes can save hours of debugging.

### `undefined is not a function`

This occurs when you try to call something that is not a function.

```javascript
const obj = {};
obj.method(); // TypeError: obj.method is not a function
```

**Solutions:** Ensure the property exists and is a function. Check for typos and asynchronous loading issues.

### `Cannot read property 'x' of undefined` / `'x' of null`

Accessing a property of `undefined` or `null` throws a TypeError.

```javascript
const user = undefined;
console.log(user.name); // TypeError: Cannot read property 'name' of undefined
```

**Solutions:** Use optional chaining (`user?.name`), default values (`user && user.name`), or check with `if (user)` before accessing.

### `null is not an object`

Similar to the above, often occurs when an expected DOM element is not found.

```javascript
const el = document.getElementById('non-existent');
el.classList.add('hidden'); // TypeError: Cannot read property 'classList' of null
```

**Solution:** Check that the element exists before manipulating it.

### `Cannot set property 'x' of undefined`

Attempting to assign to a property of `undefined`.

```javascript
let data;
data.value = 5; // TypeError: Cannot set property 'value' of undefined
```

**Solution:** Initialize the variable as an object: `let data = {};`

### `Identifier has already been declared`

Using `let` or `const` to declare a variable twice in the same scope.

```javascript
let x = 5;
let x = 10; // SyntaxError: Identifier 'x' has already been declared
```

**Solution:** Use different variable names or remove the duplicate declaration.

### `Unexpected token`

A syntax error, often due to a missing comma, bracket, or quote.

```javascript
const obj = { name: 'Alice' age: 30 }; // missing comma
```

**Solution:** Check the line and surrounding syntax carefully.

### `Maximum call stack size exceeded`

This happens with infinite recursion.

```javascript
function recurse() {
    recurse();
}
recurse(); // RangeError: Maximum call stack size exceeded
```

**Solution:** Ensure recursive functions have a proper base case.

---

## 25.6 Defensive Programming

Defensive programming is a practice of writing code that anticipates and handles potential problems gracefully. It reduces the likelihood of runtime errors and makes your code more robust.

### Validating Inputs

Always validate function parameters, especially when they come from external sources (user input, API responses, etc.).

```javascript
function greet(name) {
    if (typeof name !== 'string' || name.trim() === '') {
        console.warn('Invalid name provided');
        return 'Hello, stranger';
    }
    return `Hello, ${name.trim()}`;
}
```

### Type Checking

Use `typeof`, `instanceof`, or utility functions to ensure you're working with the expected data type.

```javascript
function processNumbers(arr) {
    if (!Array.isArray(arr)) {
        throw new TypeError('Expected an array');
    }
    // now safe to use array methods
    return arr.filter(n => typeof n === 'number').map(n => n * 2);
}
```

### Null and Undefined Checks

Before accessing properties, check that the object exists.

```javascript
// Old way
if (user && user.address && user.address.city) {
    console.log(user.address.city);
}

// Optional chaining (ES2020)
console.log(user?.address?.city ?? 'City not available');
```

### Default Parameters and Fallback Values

Provide sensible defaults to avoid `undefined` values.

```javascript
function createUser(name, age = 18, country = 'Unknown') {
    return { name, age, country };
}
```

### Using `try...catch` Around Risky Operations

Wrap code that may fail due to external factors (network, file I/O, JSON parsing) in `try...catch`.

```javascript
try {
    const data = JSON.parse(response);
} catch {
    console.error('Invalid JSON');
    data = null;
}
```

### Avoiding Magic Numbers and Strings

Use constants instead of hard‑coded values to reduce errors when values change.

```javascript
const MAX_RETRIES = 3;
const API_BASE_URL = 'https://api.example.com/v1';
```

### Writing Tests

Automated tests (unit tests, integration tests) catch errors before they reach production. While beyond the scope of this chapter, testing is a cornerstone of defensive programming.

---

## Chapter Summary

In this chapter, you learned how to handle errors and debug JavaScript code effectively:

- **Types of errors** – syntax, runtime (exceptions), and logical errors.
- **`try...catch`** – catching exceptions and using `finally` for cleanup.
- **Throwing errors** – creating and throwing custom errors to signal problems.
- **Debugging techniques** – using `console` methods, the `debugger` statement, and browser DevTools (breakpoints, network, performance, memory).
- **Common JavaScript errors** – understanding frequent pitfalls and how to fix them.
- **Defensive programming** – validating inputs, checking for null/undefined, using default values, and writing tests to prevent errors.

### Key Takeaways

- Use `try...catch` for operations that may fail due to external factors, not for everyday control flow.
- Throw meaningful errors with descriptive messages to aid debugging.
- Learn to use the browser's DevTools—they are your best friend when tracking down bugs.
- Defensive programming is about anticipating the unexpected and writing code that fails gracefully.
- Logical errors are the hardest to find; use systematic debugging and testing to uncover them.

### Practice Exercises

1. Write a function `parseJSONSafely(jsonString)` that attempts to parse JSON and returns `null` with a console warning if parsing fails. Use `try...catch`.

2. Create a custom error class `InvalidEmailError`. Write a function `validateEmail(email)` that throws this error if the email does not contain an `@` symbol. Test it with `try...catch`.

3. Take a piece of code you've written previously that lacked error handling, and refactor it to include proper validation and `try...catch` where appropriate.

4. Use the Chrome DevTools debugger to step through a recursive function (e.g., factorial) and observe the call stack and local variables at each step.

5. Write a function `safeDivide(a, b)` that returns the result of `a / b`. If `b` is zero, throw a custom `DivisionByZeroError`. In the calling code, catch the error and display a user‑friendly message.

6. Use `console.time` and `console.timeEnd` to measure the performance of two different implementations of the same task (e.g., looping vs. array methods).

7. Open a website you frequently use, open DevTools, and try to set breakpoints on event listeners to understand how the site's JavaScript works.

---

## Coming Up Next

**Chapter 26: Web Accessibility**

In the next chapter, you'll learn about making your websites usable for everyone, including people with disabilities. You'll explore semantic HTML, ARIA, keyboard navigation, and testing for accessibility—essential skills for creating inclusive web experiences.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='24. javascript_and_browser_apis.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../5. professional_practices/26. web_accessibility.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
